In [6]:
!pip install ragas langchain sentence-transformers transformers


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [7]:
!pip install ollama


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
import ollama
from ragas import evaluate
from ragas.metrics import faithfulness, answer_correctness
from langchain.schema import Document
from sentence_transformers import SentenceTransformer

print("It is starting")
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")  # Free and lightweight

print("The embedding_model is",embedding_model)

# ✅ 1. Define the coding task (Prompt for LLaMA)
code_prompt = "Write a Python function to calculate the factorial of a number recursively."

# ✅ 2. Generate code using LLaMA 3 from Ollama
print("Generating code with LLaMA 3...")
llm_response = ollama.chat(model="llama3:8b", messages=[{"role": "user", "content": code_prompt}])
llm_generated_code = llm_response["message"]["content"]

# ✅ 3. Define the Ground Truth (Expected Correct Code)
ground_truth_code = """
def factorial(n):
    if n == 0 or n == 1:
        return 1
    else:
        return n * factorial(n - 1)
"""

# Wrap code samples in RAGAS's Document format
ground_truth = Document(page_content=ground_truth_code)
generated_code = Document(page_content=llm_generated_code)

# ✅ 4. Evaluate the generated code with RAGAS
print("Evaluating generated code...")
results = evaluate(
    predictions=[generated_code],
    references=[ground_truth],
    metrics=[faithfulness, relevance, answer_correctness],
    embedding_model=embedding_model  # Use local embedding model
)

# ✅ 5. Run the generated code and verify correctness
exec_namespace = {}
try:
    exec(llm_generated_code, exec_namespace)
    result = exec_namespace 
    correct_output = result == 120
except Exception as e:
    correct_output = False
    print(f"Error in execution: {e}")

# ✅ 6. Print RAGAS scores
print("\nRAGAS Evaluation Results:")
for metric, score in results.items():
    print(f"{metric}: {score:.4f}")

# ✅ 7. Print execution correctness
print("\nExecution Correctness:", "✅ Passed" if correct_output else "❌ Failed")
